In [ ]:
%matplotlib notebook

In [ ]:
from hxrsnd import maximize_lorentz
print(maximize_lorentz.__doc__)

In [ ]:
from hxrsnd.tests import crystal, Diode
print(Diode.__doc__)

In [ ]:
#Configure RunEngine
from bluesky.run_engine import RunEngine
from bluesky.plans      import run_wrapper
from bluesky.callbacks  import LivePlot
from bluesky.utils      import install_nb_kicker
RE = RunEngine()
install_nb_kicker()

#Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [ ]:
import math

#Modify the Lorentz we are trying to maximize
actual_center      = 10.0
distribution_sigma = 2.0      #FWHM of Lorentz = 2 * sigma
distribution_amp   = distribution_sigma/1*math.pi  #Distribution maximum will be pi*amplitude / sigma
noise_multiplier   = 0.05    #Uniform noise

#Create simulated Diode readout
diode = Diode('intensity', crystal, 'angle', actual_center,
               sigma=distribution_sigma, amplitude=distribution_amp,
               noise_multiplier=noise_multiplier)

In [ ]:
#Create plans to maximize the Lorentz
bounds  = (9., 11.)
nsteps  = 10
average = 50
initial_guess = None #Guess either the center, amplitude or sigma of the Lorentz
plan = run_wrapper(maximize_lorentz(diode, crystal, 'intensity',
                                    nsteps=nsteps, bounds=bounds,
                                    average=average,
                                    position_field='angle',
                                    initial_guess=initial_guess))

In [ ]:
#Create Plot of Simulated Data
lp = LivePlot('intensity', 'angle')

In [ ]:
#Run the plan
RE(plan, lp)

In [ ]:
#Summarize success of plan
print("Final crystal position is {}, the actual center of the distribution "
      "is at {}".format(crystal.read()['angle']['value'], actual_center))
print("This leaves the diode readout at {}"
      "".format(diode.read()['intensity']['value']))